In [50]:
from Bio import SeqIO, SearchIO 
import copy
import pandas as pd

In [84]:
#SearchIO для разделения результатов поиска
file = '/home/niagara/Storage/Marfa/M_Zakirova/projects/mycofactocin/mftb/hmm_parsed_domain/parsed_hmm_per_dom_tbl.txt'
#res_domains = 
for res in SearchIO.parse(file, 'hmmscan3-domtab'):
    print(f'id: {res.id}')
    for hit in res.hits:
        print(hit.id)
#        break
        for hsp in hit.hsps:
#            print(hsp)
            print(hsp.query_range)
    break
    
 

id: WP_011566059.1
MnhB
(10, 133)
TIGR00943
(29, 135)
DUF6541
(21, 142)
K-cyclin_vir_C
(18, 51)


In [81]:
file = '/home/niagara/Storage/Marfa/M_Zakirova/projects/mycofactocin/mftb/hmm_parsed_domain/parsed_hmm_per_dom_tbl.txt'
res_domains = dict()
x = 100
for res in SearchIO.parse(file, 'hmmscan3-domtab'):
    if res.id not in res_domains:
        res_domains[res.id] = {}
    for hit in res.hits:
        for hsp in hit.hsps:
            if res_domains[res.id].keys(): #not empty
                names = copy.deepcopy(list(res_domains[res.id].keys()))
                for name in names: #проверка на перекрытие с раннее добавленными генами 
                    for q_range in res_domains[res.id][name]:
                        if (hsp.query_range[0] <= q_range[0]):
                            if (hsp.query_range[1] <= q_range[1]) and (hsp.query_range[1] - q_range[0] <= x): 
                                #перекрытие мало
                                # x - неотрицательное число, случай когда гены совсем не перекрываются соотв ситуации 
                                # hsp.query_range[1] - q_range[0] < 0 который в этом неравенстве учитывается
                                if hit.id not in res_domains[res.id]:
                                    res_domains[res.id][hit.id] = [hsp.query_range,]
                                if hsp.query_range not in res_domains[res.id][hit.id]:
                                    #для случая мультидоменного белка с одинаковыми доменами: 
                                    #один и тот же белок выравнивается на несколько мест
                                    res_domains[res.id][hit.id].append(hsp.query_range)
                            elif (hsp.query_range[1] > q_range[1]) and (q_range[1] - q_range[0] <=x):
                                #один интервал может содержаться в другом, тогда длина вложенного должна не превышать x
                                if hit.id not in res_domains[res.id]:
                                    res_domains[res.id][hit.id] = [hsp.query_range,]
                                if hsp.query_range not in res_domains[res.id][hit.id]:
                                        res_domains[res.id][hit.id].append(hsp.query_range)

                        elif hsp.query_range[0] > q_range[0]:
                            if (hsp.query_range[1] >= q_range[1]) and (q_range[1] - hsp.query_range[0]  <= x):
                                if hit.id not in res_domains[res.id]:
                                    res_domains[res.id][hit.id] = [hsp.query_range,]
                                if hsp.query_range not in res_domains[res.id][hit.id]:
                                    res_domains[res.id][hit.id].append(hsp.query_range)
                            elif (hsp.query_range[1] < q_range[1]) and (hsp.query_range[1] - hsp.query_range[0] <=x):
                                if hit.id not in res_domains[res.id]:
                                    res_domains[res.id][hit.id] = [hsp.query_range,]
                                if hsp.query_range not in res_domains[res.id][hit.id]:
                                    res_domains[res.id][hit.id].append(hsp.query_range)
                                                            
            else:
                res_domains[res.id][hit.id] = [hsp.query_range,]
                
                                
            
res_domains

{'WP_011566059.1': {'MnhB': [(10, 133)], 'K-cyclin_vir_C': [(18, 51)]},
 'WP_011566060.1': {'TIGR00940': [(48, 754)],
  'Proton_antipo_M': [(677, 696)],
  'TIGR01972': [(49, 465)],
  'TIGR01770': [(73, 427), (642, 651)],
  'TIGR01960': [(17, 444)],
  'TIGR00944': [(36, 450)],
  'MbhE': [(679, 758)],
  'MbhD': [(26, 37),
   (66, 83),
   (154, 173),
   (280, 299),
   (491, 504),
   (598, 661),
   (674, 699),
   (743, 750)],
  'Proton_antipo_N': [(54, 91), (154, 170)]},
 'WP_011566061.1': {'TIGR03965': [(213, 248), (261, 734)],
  'Glycos_transf_2': [(345, 457)],
  'Glyco_tranf_2_3': [(343, 558)],
  'Methyltransf_23': [(41, 76), (103, 212)],
  'Methyltransf_12': [(115, 207)],
  'Methyltransf_11': [(115, 209)],
  'Methyltransf_25': [(114, 205)],
  'TIGR03937': [(339, 431), (502, 543)],
  'TIGR01983': [(112, 209)],
  'Glyco_tranf_2_2': [(371, 433), (501, 530)],
  'TIGR04283': [(345, 443), (502, 532)],
  'TIGR03469': [(336, 385)],
  'Glyco_trans_2_3': [(423, 564)],
  'Methyltransf_31': [(112,

In [82]:
domains_names = {}
for key, value in res_domains.items():
    domains_names[key] = list(value.keys())

domains_names = pd.DataFrame({'domains': domains_names.values()}, index =res_domains.keys())
domains_names

,domains
WP_011566059.1,"[MnhB, K-cyclin_vir_C]"
WP_011566060.1,"[TIGR00940, Proton_antipo_M, TIGR01972, TIGR01..."
WP_011566061.1,"[TIGR03965, Glycos_transf_2, Glyco_tranf_2_3, ..."
WP_011566062.1,[Creatininase]
WP_011566063.1,"[TIGR03966, TIGR02151, TIGR01302, IMPDH, Glu_s..."
...,...
WP_160900598.1,"[DoxX_3, DoxX_2, DoxX, DUF2892]"
WP_160901386.1,[DNA_alkylation]
WP_160900597.1,"[Pyr_redox_2, TIGR01350, TIGR03452, Pyr_redox,..."
WP_160900596.1,"[FAM70, DUF3040]"


In [83]:
#domains_names.to_csv('/home/niagara/Storage/Marfa/M_Zakirova/projects/mycofactocin/mftb/hmm_parsed_domain/mftb_domains_names_100.csv')

---

Ниже принты с прошлого запуска, для поиска ошибки, оставила так как удобно видеть какие данные лежат в hit и hsp

In [47]:
print(hit)

Query: WP_072738333.1
       <unknown description>
  Hit: Bromo_TP (77)
       Bromodomain associated
 HSPs: ----  --------  ---------  ------  ---------------  ---------------------
          #   E-value  Bit score    Span      Query range              Hit range
       ----  --------  ---------  ------  ---------------  ---------------------
          0      0.41      11.10       ?        [180:226]                 [4:53]


In [49]:
print(hsp)

      Query: WP_072738333.1 <unknown description>
        Hit: Bromo_TP Bromodomain associated
Query range: [180:226] (0)
  Hit range: [4:53] (0)
Quick stats: evalue 0.41; bitscore 11.10
  Fragments: 1 (? columns)


In [50]:
print(res)

Program: <unknown program> (<unknown version>)
  Query: WP_072738333.1 (239)
         <unknown description>
 Target: <unknown target>
   Hits: ----  -----  ----------------------------------------------------------
            #  # HSP  ID + description
         ----  -----  ----------------------------------------------------------
            0      1  DNA_alkylation  DNA alkylation repair enzyme
            1      3  HEAT_2  HEAT repeats
            2      1  Bromo_TP  Bromodomain associated
